In [ ]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
from torch.distributions import MultivariateNormal

In [ ]:
# Estableciendo los hiperparámetros
hidden_dim = 256
lr_actor = 3e-4
lr_critic = 1e-3
gamma = 0.99
gae_lambda = 0.95
ppo_epochs = 10
mini_batch_size = 64
ppo_clip = 0.2
buffer_size = 2048
update_timestep = buffer_size
action_std = 0.5  # Standard deviation for action exploration

In [ ]:
# Construyendo la red actor-crítico
class ActorCritic(nn.Module):

    def __init__(self, num_actions):
        super(ActorCritic, self).__init__()
        # capas comunes
        self.conv1 = nn.Conv2d(3, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.fc1 = nn.Linear(64 * 7 * 7, hidden_dim)
        # capas de actor
        self.fc_actor = nn.Linear(hidden_dim, num_actions)
        self.log_std = nn.Parameter(torch.zeros(num_actions))
        # capas críticas
        self.fc_critic = nn.Linear(hidden_dim, 1)

    def forward(self, state):
        x = F.relu(self.conv1(state))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        # Actor
        action_mean = self.fc_actor(x)
        action_var = torch.exp(self.log_std.expand_as(action_mean))
        cov_mat = torch.diag_embed(action_var)
        # Critic
        value = self.fc_critic(x)
        return action_mean, cov_mat, value

In [ ]:
# Implementación del búfer de memoria
class Memory:

    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []

    def clear_memory(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]

In [ ]:
# Construyendo el Agente PPO
class PPO:

    def __init__(self, num_actions):
        self.policy = ActorCritic(num_actions)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr_actor)
        self.policy_old = ActorCritic(num_actions)
        self.policy_old.load_state_dict(self.policy.state_dict())
        self.MseLoss = nn.MSELoss()

    def select_action(self, state):
        with torch.no_grad():
            action_mean, action_var, _ = self.policy_old(state)
            dist = MultivariateNormal(action_mean, action_var)
            action = dist.sample()
            action_logprob = dist.log_prob(action)
        return action.detach().numpy(), action_logprob.detach()

    def update(self, memory):
        # Estimación Monte Carlo de las recompensas
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(memory.rewards), reversed(memory.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
        # Normalizar las recoompensas
        rewards = torch.tensor(rewards, dtype=torch.float32)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-7)
        # Convertir la lista a tensor
        old_states = torch.stack(memory.states).detach()
        old_actions = torch.stack(memory.actions).detach()
        old_logprobs = torch.stack(memory.logprobs).detach()
        # Optimizar la política para k épocas
        for _ in range(ppo_epochs):
            # Evaluar acciones previas
            action_means, action_vars, state_values = self.policy(old_states)
            dists = MultivariateNormal(action_means, action_vars)
            logprobs = dists.log_prob(old_actions)
            dist_entropy = -logprobs.mean()
            # Encontrar el ratio (pi_theta / pi_theta__old)
            ratios = torch.exp(logprobs - old_logprobs.detach())
            # Encontrar Surrogate Loss
            advantages = rewards - state_values.detach()
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-ppo_clip, 1+ppo_clip) * advantages
            loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
            # Obtener gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
        # Copiar nuevos pesos a la nueva política
        self.policy_old.load_state_dict(self.policy.state_dict())


In [ ]:
# Preprocesamiento de los estados
def preprocess_state(state):
    state = np.ascontiguousarray(state, dtype=np.float32) / 255
    state = torch.from_numpy(state)
    return state.unsqueeze(0)

In [ ]:
# Configurar el entorno
env = gym.make('CarRacing-v2')

In [ ]:
# Creando la memoria
memory = Memory()

In [ ]:
# Creando el agente
ppo = PPO(env.action_space.shape[0])

In [ ]:
# Implementación del circuito de entrenamiento
state = env.reset()
state = preprocess_state(state)
for t in range(1, update_timestep+1):
    action, action_logprob = ppo.select_action(state)
    next_state, reward, done, _ = env.step(action)
    next_state = preprocess_state(next_state)
    memory.states.append(state)
    memory.actions.append(torch.tensor(action))
    memory.logprobs.append(action_logprob)
    memory.rewards.append(reward)
    memory.is_terminals.append(done)
    state = next_state
    if done:
        state = env.reset()
        state = preprocess_state(state)
    if t % update_timestep == 0:
        ppo.update(memory)
        memory.clear_memory()